<a href="https://colab.research.google.com/github/SeojinSeojin/AAI3005/blob/main/scrap_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt install -y chromium-chromedriver

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [589 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,455 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,389 kB]
Get:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,415 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages

In [3]:
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import csv

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


In [6]:
csv_input_file = 'scraped_data.csv'


In [7]:
import re

namePattern = r"(\w+)\((\w+)/(\w+)\)"
linkPattern = r"dept_cd=(\d+)"

In [8]:
from tqdm.notebook import tqdm

In [9]:
start_row = 0
total_row = 1000

In [10]:
csv_output_file = f'scraped_data_3_{start_row}_{start_row+total_row}.csv'

In [ ]:
pbar = tqdm(total=total_row, desc="Scraping Progress")

with open(csv_input_file, 'r') as infile, open(csv_output_file, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Chrome driver 설정
    driver = webdriver.Chrome(options=chrome_options)

    for idx, row in enumerate(reader):
        if idx < start_row:
            # 시작 행에 도달할 때까지 작업을 건너뜁니다.
            continue
        if idx >= start_row + total_row:
            # 마지막 행에 도달하면 루프를 빠져나옵니다.
            break
        if(len(row)<9):
          pbar.update(1)
          continue
        column_value = row[9]  # 10번째 컬럼

        # 페이지 로딩
        url = f"https://likms.assembly.go.kr/bill/coactorListPopup.do?billId={column_value}"
        driver.get(url)
        cells = driver.find_elements(By.TAG_NAME, "a")
        for cell in cells:
          result_row = [column_value]
          # 스크래핑한 값을 별도의 CSV 파일에 저장
          text = cell.text
          parsedText = re.match(namePattern, text)
          if parsedText:
            result_row.append(parsedText.group(1))
            result_row.append(parsedText.group(2))
            result_row.append(parsedText.group(3))
          try:
            href = cell.get_attribute("href")
            parsedHref = re.search(linkPattern, href)
            if parsedHref:
              result_row.append(parsedHref.group(1))
          except:
            print(f"cannot execute re.search on {href}")
          writer.writerow(result_row)
        pbar.update(1)

    # 드라이버 종료
    driver.quit()


Scraping Progress:   0%|          | 0/1000 [00:00<?, ?it/s]

cannot execute re.search on None
